<a href="https://colab.research.google.com/github/machine-learning-airu-ozone/ChE_Utah_AirQuality_ML_Ozone/blob/master/Bayesian_Hyperparameterization_Neural_Network_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Neural Network Notebook Optimization-Bayesian Template
## Summer 2019-Air U Project 
### Timothy Quah

This template is an example of how to train neural networks on Google Colab. Google Colab has free GPU/TPU resources that can help train neural networks that deal with large datasets

###  Load Data into Google Colab

First thing we must do is to load our data into the Google Colab environment. To do this we must do the following:
First we need to mount the drive which we use the following lines to do. There are ways to automate this process, but I am honestly a bit too lazy to do this.

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


The next step we will do is to verify that the path exist to our files that we have stored on google drive. 

In [0]:
import os
import_data_path = '/content/drive/My Drive/AirQuality_Research/Remove_Params_Upload_GoogleDrive/SWD/'


print('Does Data Path? '+str(os.path.exists(import_data_path)))


Does Data Path? True
Does Function Path? True


Next we need to clone our repository and then load it into this notebook

In [0]:
Repository_Path = '/content/drive/My Drive/AirQuality_Research/Repository'
print('Does Repo Path? '+str(os.path.exists(Repository_Path)))
os.chdir(Repository_Path)


if len(os.listdir('/content/drive/My Drive/AirQuality_Research/Repository'))>0:
  !rm -r *
  print('Old Repository Deleted')
  
! git clone https://github.com/machine-learning-airu-ozone/ChE_Utah_AirQuality_ML_Ozone
import_script_path = '/content/drive/My Drive/AirQuality_Research/Repository/ChE_Utah_AirQuality_ML_Ozone/Functions'
print('Does Function Path? '+str(os.path.exists(import_script_path)))

Does Repo Path? True
Old Repository Deleted
Cloning into 'ChE_Utah_AirQuality_ML_Ozone'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 40 (delta 8), reused 28 (delta 3), pack-reused 0
Unpacking objects: 100% (40/40), done.
ChE_Utah_AirQuality_ML_Ozone
Does Function Path? True


Next we import all the packages we need. If you need a specific version it is possible, but will take an extra line: example  !pip install seaborn==0.9.0


In [0]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras.optimizers as optimizers
from keras.models import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt
import random
from copy import deepcopy
from keras import backend as K
from sklearn.preprocessing import MinMaxScaler
from functools import partial


Using TensorFlow backend.


We need to install bayesian optimization we do this using the following method

In [0]:
!pip install bayesian-optimization
from bayes_opt import BayesianOptimization

Now we need to load some scripts from our custom made functions into the enviroment

In [0]:
os.chdir(import_script_path)
from Trainer_Functions import r2_keras,model_neural_network,load_evaluate_neural_net,norm_divider,divider_XY,mse,r2


Next we need to just make sure to close all plots and just in case we have set random seeds


In [0]:
plt.close('all')
random.seed(7)
np.random.seed(7)
load_data_list = os.listdir(import_data_path)
print(load_data_list)

['outsider_data.csv', 'train_validate_data.csv']


We see in this case we have two files one called 'All_Data_norm.csv' we will load both files (including 'train_validate_data.csv') now

In [0]:
i = 1
full_path = os.path.join(import_data_path,load_data_list[i])
df = pd.read_csv(full_path)
header =list(df) 


Now we need to divide the training data from the validation data as well as inputs from outputs. In this case we will set up 70% training and 30% Validation.


---



In [0]:
header_num = len(header)
Full_List = list(np.arange(0,header_num-1+1e-6,1,dtype=int))
Y_Loc =  header.index('O3 Value')
Y_header_list = []
Y_header_list.append(Y_Loc)
X_header_list = list(set(Full_List)-set(Y_header_list))

data_array = np.array(df)
train_list,valid_list = norm_divider(data_array)
X,Y,X_valid,Y_valid = divider_XY(X_header_list,Y_header_list,data_array,train_list,valid_list)

Now we need to prepare the data in the other dataset


In [0]:
i = 0
full_path = os.path.join(import_data_path,load_data_list[i])
df_test = pd.read_csv(full_path)

other_header = list(df_test)
for i in range(0,len(delete_list)):
  index = other_header.index(delete_list[i])
  del df_test[other_header[index]]
  del other_header[index]

data_array_test = np.array(df_test)
X_Other = data_array_test[:,X_header_list]
Y_Other = data_array_test[:,Y_header_list]


We now need to create an optimization function pay careful attention to datatypes and that we are maximizing not minimizing thus we take the negative of error

In [0]:
def all_optimize_function(X,X_valid,X_other,X_header_list,\
                         Y,Y_valid,Y_other,Y_header_list,\
                         errorweight,return_model,verbose_set,\
                         layers,nodes,droprate,\
                         learning_rate,batchsize,epoch):
    layers = int(layers)
    nodes = int(nodes)
    batchsize = int(batchsize)
    epoch = int(epoch)
    input_dim_ = len(X_header_list)
    output_dim_ = len(Y_header_list)
    model = model_neural_network(layers,nodes,input_dim_,output_dim_,\
                                 DropPercent = droprate)
    optimizers.Adam(lr=learning_rate)
    model.compile(loss='mean_squared_error',optimizer="adam", metrics=[r2_keras])
    history = model.fit(x=X, y = Y, nb_epoch=epoch, batch_size=batchsize,\
                        verbose=verbose_set)
    Y_pred = model.predict(X_valid)
    Valid_MSE = mse(Y_valid,Y_pred)
    Y_other_pred = model.predict(X_other)
    Other_MSE = mse(Y_other,Y_other_pred)
    if return_model:
        return -Other_MSE,model,history
    else:
        return -Other_MSE

Now we need to set bounds on the parameters we are optimizing

In [0]:
optimize_function = partial(all_optimize_function,X,X_valid,X_Other,X_header_list,\
                          Y,Y_valid,Y_Other,Y_header_list,errorweight = 1/3,\
                          return_model=False,verbose_set = 0)

pbounds = {'layers':(1,5),'nodes':(10,50),'droprate':(0.01,0.2),\
       'learning_rate':(0.01,0.5),'batchsize':(20,80),'epoch':(50,100)}

optimizer = BayesianOptimization(f=optimize_function,pbounds=pbounds,random_state=1,)


Now lets do the optimization 

In [0]:
points = 3
itterations = 5
optimizer.maximize(init_points=points,n_iter=itterations,)
print(optimizer.max)

W0725 19:53:14.335430 140177552824192 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0725 19:53:14.353483 140177552824192 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0725 19:53:14.356034 140177552824192 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



|   iter    |  target   | batchsize | droprate  |   epoch   |  layers   | learni... |   nodes   |
-------------------------------------------------------------------------------------------------


W0725 19:53:14.374746 140177552824192 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0725 19:53:14.384134 140177552824192 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0725 19:53:14.530088 140177552824192 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: The `nb_epoch` argument in `fit` has been renamed `

|  1        | -0.0204   |  45.02    |  0.1469   |  50.01    |  5.116    |  0.08191  |  36.46    |
|  2        | -0.01802  |  31.18    |  0.07566  |  69.84    |  6.772    |  0.2154   |  77.97    |
|  3        | -0.0198   |  32.27    |  0.1768   |  51.37    |  7.693    |  0.2145   |  69.11    |


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  # This is added back by InteractiveShellApp.init_path()


...

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  # This is added back by InteractiveShellApp.init_path()


|  12       | -0.01649  |  78.1     |  0.08109  |  99.99    |  9.123    |  0.1315   |  97.85    |


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  # This is added back by InteractiveShellApp.init_path()


|  13       | -0.0146   |  21.0     |  0.01365  |  99.32    |  9.564    |  0.01317  |  98.34    |


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  # This is added back by InteractiveShellApp.init_path()
